# Dataset pipeline for Early Convnet Approach (Exploration)

Possible leads:
* https://stackoverflow.com/questions/54745903/how-to-implement-an-image2d-array-sequence-sliding-window-in-tensorflow
* https://stackoverflow.com/questions/55429307/how-to-use-windows-created-by-the-dataset-window-method-in-tensorflow-2-0s
* https://stackoverflow.com/questions/38235643/getting-started-with-tensorflow-split-image-into-sub-imagess

Goal:s
> every 40 × 40 window in the input stepped every 4 pixels

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import src.data.datasets.deep_globe_2018
import PIL

ds, ds_info = tfds.load(
        # Useful link: https://github.com/tensorflow/datasets/issues/2680#issuecomment-778801923
        name='deep_globe_2018',
        download=False,
        with_info=True,
    )

ds_info

2023-12-26 20:54:34.944877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-26 20:54:42.246840: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


tfds.core.DatasetInfo(
    name='deep_globe_2018',
    full_name='deep_globe_2018/1.0.0',
    description="""
    TODO(deep_globe_2018): Markdown description of that will appear on the catalog page.
    Description is **formatted** as markdown.
    
    It should also contain any processing which has been applied (if any),
    (e.g. corrupted example skipped, images cropped,...):
    """,
    homepage='https://dataset-homepage/',
    data_path='/root/tensorflow_datasets/deep_globe_2018/1.0.0',
    file_format=tfrecord,
    download_size=Unknown size,
    dataset_size=1.95 GiB,
    features=FeaturesDict({
        'file_name': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'segmentation_mask': Image(shape=(None, None, 1), dtype=uint8),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'all_images': <SplitInfo num_examples=803, num_shards=16>,
    },
    citation="""// TODO(deep_globe_2018):

In [22]:
def load_image(datapoint):    
    input_image = tf.image.resize(datapoint['image'], (512, 512))
    
    input_mask = tf.image.resize(
        datapoint['segmentation_mask'],
        (512, 512),
        method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
    )
    
    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

In [4]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [5]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [34]:
images = ds['all_images'].take(10).map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

tensorflow.python.data.ops.map_op._ParallelMapDataset

In [30]:
image_batches = (
    images
    .cache() # The first time the dataset is iterated over, its elements will be cached either in the specified file or in memory. Subsequent iterations will use the cached data.
    .window( size=40, shift=4, drop_remainder=True ).flat_map( lambda x, y: tf.data.Dataset.zip( (x.batch(40), y.batch(40)) ) )
    # .batch(1) # Combines consecutive elements of this dataset into batches.
    .prefetch(buffer_size=tf.data.AUTOTUNE))

for i, m in image_batches.take(2):
    sample_image, sample_mask = i[0], m[0]
    display([sample_image, sample_mask])